# Affirmations — LangChain + LangGraph + Ollama

This notebook demonstrates generating structured affirmations for spiritual practices using:

- **Ollama** (local Gemma 3 4B model) — fully local inference, no API keys for the LLM
- **LangChain LCEL** — simple `prompt | model | parser` chains
- **LangGraph ReAct agent** — research-augmented generation with tool calling
- **Research Tools**: DuckDuckGo, SearxNG (self-hosted metasearch), Wikipedia, Jina Reader (optional)
- **Trafilatura** — best-in-class HTML content extraction for the research processing layer
- **Pydantic v2** — structured output validation

## Prerequisites

1. Run `nx run affirmations:setup` to start Ollama + SearxNG and pull `gemma3:4b`
2. Ollama API available at `http://localhost:11434`
3. SearxNG available at `http://localhost:8889`
4. Optional: Set `JINA_API_KEY` environment variable to enable Jina Reader tool

## Architecture

```
User Query
    │
    ▼
LangGraph ReAct Agent
    │
    ├─► web_search (DuckDuckGo) ──┐
    ├─► searxng_search           ├─► research.py (Trafilatura) ──► Context Budget
    ├─► wikipedia_lookup ────────┘
    └─► jina_reader (optional)
    │
    ▼
Structured Affirmation (Pydantic)
```

## 1. Imports & LLM Setup

In [ ]:
import sys
sys.path.insert(0, '..')

from src.llm import create_llm
from src.models import Affirmation, AffirmationSet
from src.tools import create_tools
from src.chains import create_affirmation_chain
from src.agent import create_research_agent
from src.output import save_affirmations, load_affirmations
from src.practices import PRACTICES
from src.research import process_search_results, extract_content
from pathlib import Path

# Create the LLM — connects to local Ollama at http://localhost:11434
llm = create_llm(model='gemma3:4b', temperature=0.7)
print(f'LLM: {llm.model} at {llm.base_url}')

## 2. Create Research Tools

In [ ]:
tools = create_tools()

print(f'Available tools ({len(tools)} total):')
for tool in tools:
    print(f'  • {tool.name}: {tool.description[:80]}...')

## 3. Research Processing Layer Demo

The research processing layer transforms raw HTML/text from search tools into clean, LLM-digestible context using Trafilatura.

In [ ]:
# Simulate raw HTML from a search result
raw_html = '''
<html>
<nav>Home | About | Contact</nav>
<header>My Site</header>
<main>
  <article>
    <h1>The Tower Tarot Card Meaning</h1>
    <p>The Tower card (XVI) represents sudden change, upheaval, and revelation.
    In the Rider-Waite deck, it depicts a tower struck by lightning with figures falling.
    Despite its dramatic imagery, The Tower ultimately brings liberation from false structures.</p>
    <p>Upright meanings: sudden change, upheaval, chaos, revelation, awakening.
    Reversed meanings: fear of change, averting disaster, delaying the inevitable.</p>
  </article>
</main>
<footer>Ads | Newsletter | Terms</footer>
</html>
'''

print('=== Raw HTML (first 300 chars) ===')
print(raw_html[:300])

print('\n=== Trafilatura-extracted content ===')
extracted = extract_content(raw_html)
print(extracted)

print('\n=== Final processed research result ===')
processed = process_search_results(raw_html, 'Tower tarot card meaning', 'wikipedia')
print(processed)

## 4. Simple Chain (No Research Tools)

Direct generation using only the LLM's training data — fast but less contextually rich.

In [ ]:
# Simple LCEL chain: prompt | llm.with_structured_output(Affirmation)
chain = create_affirmation_chain(llm)

# Generate a single affirmation for The Tower tarot card
affirmation = chain.invoke({
    'practice': 'tarot',
    'topic': 'The Tower',
    'structure': 'I am [positive quality] through [transformative process]'
})

print('Generated Affirmation (Simple Chain):')
print(f'  Text: {affirmation.text}')
print(f'  Practice: {affirmation.practice}')
print(f'  Structure: {affirmation.structure}')
print(f'  Keywords: {affirmation.keywords}')

## 5. Research Agent (With Tools)

The LangGraph ReAct agent researches the topic first, then generates an affirmation informed by the processed research context.

In [ ]:
# Create the research agent
agent = create_research_agent(llm, tools)

print('Research agent created.')
print(f'Graph nodes: {list(agent.get_graph().nodes.keys())}')

In [ ]:
from langchain_core.messages import HumanMessage

# Invoke the agent for a Tower tarot affirmation
# The agent will research first, then generate
user_request = (
    'Research The Tower tarot card (XVI) — its symbolism, history, and meaning. '
    'Then generate an affirmation for someone working with The Tower energy, '
    'using the structure: "I am [positive quality] through [transformative process]"'
)

print('Invoking research agent...')
print(f'Request: {user_request[:100]}...')
print()

result = agent.invoke({'messages': [HumanMessage(content=user_request)]})

# Display the agent's reasoning trace
print('=== Agent Reasoning Trace ===')
for i, msg in enumerate(result['messages']):
    msg_type = type(msg).__name__
    content_preview = str(msg.content)[:200] if msg.content else '(tool call)'
    print(f'[{i}] {msg_type}: {content_preview}')
    print()

print('=== Final Agent Response ===')
print(result['messages'][-1].content)

## 6. SearxNG Engine Targeting

In [ ]:
# Demonstrate SearxNG engine-specific targeting
from langchain_community.utilities import SearxSearchWrapper

searx = SearxSearchWrapper(searx_host='http://localhost:8889')

# Target Wikipedia specifically for authoritative content
print('=== SearxNG: Wikipedia engine targeting ===')
try:
    wiki_results = searx.run('tarot Major Arcana symbolism', engines=['wikipedia'])
    print(wiki_results[:500])
except Exception as e:
    print(f'SearxNG not available: {e}')

# Target ArXiv for academic/scientific sources
print('\n=== SearxNG: ArXiv engine targeting (science category) ===')
try:
    arxiv_results = searx.run('spiritual symbolism psychology', categories='science')
    print(arxiv_results[:500])
except Exception as e:
    print(f'SearxNG not available: {e}')

## 7. Batch Generation for a Practice

In [ ]:
# Generate affirmations for a few chakras using the simple chain
chain = create_affirmation_chain(llm)
chakra_practice = PRACTICES['chakras']

affirmations_list = []
# Generate for just the first 3 chakras to keep the demo fast
for topic in chakra_practice.topics[:3]:
    structure = chakra_practice.structures[0]
    print(f'Generating affirmation for: {topic}...')
    aff = chain.invoke({'practice': 'chakras', 'topic': topic, 'structure': structure})
    affirmations_list.append(aff)
    print(f'  → {aff.text}')

# Create an AffirmationSet
chakra_set = AffirmationSet(practice='chakras', affirmations=affirmations_list)
print(f'\nGenerated {len(chakra_set.affirmations)} chakra affirmations')

## 8. Save & Load Results

In [ ]:
# Save to output/chakras.json
output_dir = Path('../output')
saved_path = save_affirmations(chakra_set, output_dir)
print(f'Saved to: {saved_path}')

# Read back and display
loaded = load_affirmations('chakras', output_dir)
print(f'\nLoaded {len(loaded.affirmations)} affirmations for {loaded.practice}:')
for aff in loaded.affirmations:
    print(f'  • {aff.text}')
    print(f'    Keywords: {aff.keywords}')

## 9. Available Practices

Explore all configured spiritual practices:

In [ ]:
print(f'Configured practices: {list(PRACTICES.keys())}\n')
for name, config in PRACTICES.items():
    print(f'📿 {name.upper()} ({len(config.topics)} topics)')
    print(f'   Topics: {config.topics[:3]} ...')
    print(f'   Structures: {config.structures[0]}')
    print()